<a href="https://colab.research.google.com/github/vectominist/MiniASR/blob/main/example/example_librispeech_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MiniASR Tutorial: LibriSpeech Training**
This is a tutorial for training an end-to-end automatic speech recognition model with the toolkit [MiniASR](https://github.com/vectominist/MiniASR).  
You can run this notebook on [Google Colab](colab.research.google.com/), but to train an ASR model completely requires a Pro account since it needs several hours to converge.

## **Download Code & Install Dependencies**
Ref: [MiniASR](https://github.com/vectominist/MiniASR)

In [ ]:
! git clone https://github.com/vectominist/MiniASR.git
% cd MiniASR

In [ ]:
! pip3 install -e ./

## **Download Data**
- training set: [Libri-light](https://github.com/facebookresearch/libri-light) fine-tuning set (10 hours, 0.6G)
- development set: [LibriSpeech](https://www.openslr.org/12) `dev-clean` set
- testing set: [LibriSpeech](https://www.openslr.org/12) `test-clean` set

In [ ]:
! mkdir -p data
% cd data
! wget https://dl.fbaipublicfiles.com/librilight/data/librispeech_finetuning.tgz
! tar zxf librispeech_finetuning.tgz
! rm librispeech_finetuning.tgz

In [ ]:
! wget https://www.openslr.org/resources/12/dev-clean.tar.gz
! wget https://www.openslr.org/resources/12/test-clean.tar.gz
! tar zxf dev-clean.tar.gz
! tar zxf test-clean.tar.gz
! rm dev-clean.tar.gz
! rm test-clean.tar.gz
% cd ..

## **Preprocess Data**
Find all data in the corpus and extract vocabularies. We use characters as text tokens since the dataset is small.

In [ ]:
# Train set
! miniasr-preprocess \
        -c LibriSpeech \
        -p data/librispeech_finetuning \
        -s 1h \
        -o data/libri_train_1h \
        --gen-vocab \
        --char-vocab-size 40

! miniasr-preprocess \
        -c LibriSpeech \
        -p data/librispeech_finetuning \
        -s 9h \
        -o data/libri_train_9h

# Development set
! miniasr-preprocess \
        -c LibriSpeech \
        -p data/LibriSpeech \
        -s dev-clean \
        -o data/libri_dev

# Test set
! miniasr-preprocess \
        -c LibriSpeech \
        -p data/LibriSpeech \
        -s test-clean \
        -o data/libri_test

## **Training**
- Modify `MiniASR/egs/librispeech/config/ctc_train_example.yaml` for changing training hyper-parameters.
- The results will be saved to `MiniASR/model/ctc_libri-10h_char`.

In [ ]:
! mkdir -p model

In [ ]:
! minasr-asr --config egs/librispeech/config/ctc_train_example.yaml

# Resume training with this command:
# ! minasr-asr --ckpt model/ctc_libri-10h_char/epoch=4-step=429.ckpt

10-01 12:12 run_asr.py.main(99) Training mode.
10-01 12:12 dataloader.py.create_dataloader(86) Creating text tokenizer of character level.
10-01 12:12 dataloader.py.create_dataloader(91) Generating datasets and dataloaders. (mode = train)
10-01 12:12 dataset.py.__init__(28) Loading data from ['data/libri_train_1h/data_list_sorted.json', 'data/libri_train_9h/data_list_sorted.json']
100% 2763/2763 [00:00<00:00, 12494.93it/s]
10-01 12:12 dataset.py.__init__(49) 2763 audio files found (mode = train)
10-01 12:12 dataset.py.__init__(28) Loading data from ['data/libri_dev/data_list_sorted.json']
100% 2703/2703 [00:00<00:00, 21381.83it/s]
10-01 12:12 dataset.py.__init__(49) 2703 audio files found (mode = dev)
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that e

## **Testing**
- Specify your checkpoint with `--ckpt`.

In [ ]:
! minasr-asr \
    --config egs/librispeech/config/ctc_test_example.yaml \
    --test \
    --override "args.data.dev_paths=['data/libri_test/data_list_sorted.json']" \
    --ckpt model/ctc_libri-10h_char/epoch=44-step=3869.ckpt

10-01 13:28 basic_setups.py.override(89) Override: args.data.dev_paths = ['data/libri_test/data_list_sorted.json']
10-01 13:28 run_asr.py.main(105) Testing mode.
[s3prl.upstream.experts] Warning: can not import s3prl.upstream.wav2vec.expert: No module named 'fairseq'. Pass.
[s3prl.upstream.experts] Warning: can not import s3prl.upstream.wav2vec2_hug.expert: No module named 'transformers'. Pass.
[s3prl.upstream.experts] Warning: can not import s3prl.upstream.roberta.expert: No module named 'fairseq'. Pass.
[s3prl.upstream.experts] Warning: can not import s3prl.upstream.vq_wav2vec.expert: No module named 'fairseq'. Pass.
[s3prl.upstream.experts] Warning: can not import s3prl.upstream.wav2vec2.expert: No module named 'fairseq'. Pass.
[s3prl.upstream.experts] Warning: can not import s3prl.upstream.hubert.expert: No module named 'fairseq'. Pass.
[s3prl.upstream.experts] Warning: can not import s3prl.upstream.decoar2.expert: No module named 'fairseq'. Pass.
[s3prl.hub] Warning: can not impor

## **Inference**

In [ ]:
from miniasr.utils import load_from_checkpoint, sequence_distance
from miniasr.data.audio import load_waveform

model, args, tokenizer = load_from_checkpoint(
    'model/ctc_libri-10h_char/epoch=44-step=3869.ckpt', 'cuda')
waves = [load_waveform('data/LibriSpeech/dev-clean/6345/93302/6345-93302-0025.flac').to('cuda')]
hyps = model.recognize(waves)

In [ ]:
print(hyps[0])
ref = 'ARE YOU REALLY GOING TO THROW ME OVER FOR A THING LIKE THIS'
res_cer = sequence_distance(ref, hyps[0], mode='char')
res_wer = sequence_distance(ref, hyps[0], mode='word')
print('CER = {:.2f}%'.format(100. * res_cer['distance'] / res_cer['length']))
print('WER = {:.2f}%'.format(100. * res_wer['distance'] / res_wer['length']))

Y WILY O T OE ME R THING MY FES
CER = 59.32%
WER = 84.62%
